# IL027 Interdisciplinary Computer Modelling

## Lecture 4 - N-body Problem and Mission to Mars

### James Kermode - School of Engineering

**Run this code before running the remaining code**

In [ ]:
using Test
using Plots
using ForwardDiff
using Printf, LinearAlgebra
gr(fmt=:png)

## Background

For the purposes of this lecture, imagine we are in the year 2044 and human colonies are already established on Mars. There are some aliens that want to attack and conquer Mars and the Earth.

<img src="img/Alien_attack.jpg" alt="Alien_attack" style="width: 400px;" align="middle" />

## Escape velocity of a ballistic missile

The UK Space Agency (UKSA) will launch a spacecraft to defend the Earth from aliens. 

<img src="img/UKSA.jpg" alt="UKSA" style="width: 300px;" align="middle"/>

Engineers need to calculate the minimum velocity (called escape velocity $v_e$) with which the missile must be launched in order to escape the Earth. Assuming the Earth is a spherical symmetric massive body with no atmosphere, the escape velocity $v_e$ of any object launched on Earth’s surface is:

$$ v_e = \sqrt{\frac{2GM_e}{R_e}} $$

where $G$ is the universal gravitational constant, $M_e$ the mass of the Earth, and $R_e$ is the radius of the Earth. Note that this equation is independent of the mass of the object. The following sketch illustrates the idea of launching an object to space at different velocities:

<img src="img/Escape_velocity.jpg" alt="Escape_velocity" style="width: 300px;" align="middle"/>

**Lecture Question 1.** Look up the values of $G$, $M_e$ and $R_e$, and then calculate $v_e$.

In [ ]:
G = 6.67408e-11 # m^3 kg^{-1} s^{-1}
M_e = 5.972e24 # kg
R_e = 6.373e6 # m

v_e = sqrt(2*G*M_e/R_e) # m/s

@printf("Escape velocity: %.1f km/s", v_e / 1e3) # printf macro useful for formatted printing

## Flying a spacecraft to mars

As a response strategy, the UKSA decided to send a spacecraft to Mars to defend the human colonies there. To do this, it is necessary to know the trajectories of Earth and Mars with respect to the Solar System frame of reference. Trajectories of planets and stars are inter-dependent, since they mutually attract each other due to gravity. This makes the process of prediction a non-trivial task.

### 3-body problem

A group of student interns at UKSA decide to model the trajectories of Earth and Mars using Newtonian gravity. They make the following assumptions: 

- the planets are moving in a two-dimentional plane, 
- the effect of the rest of the planets of the Solar System on the trajectories of the Earth and Mars is negligible. 

Any object subjected to a gravitational field has potential energy $U$:

$$ U = -G \frac{m_1 m_2}{r_{12}} $$

where $G$ is the universal gravitational constant, $m_1$ the mass of the object creating a gravitational field, $m_2$ the mass of the object subjected to the gravitational field and $r_{12}$ is the distance between the objects. In this case, the Earth is subjected to the gravitational fields of the Sun and Mars, and vice versa. The total potential energy with $N$ bodies is

$$ U(\mathbf{R}) = -\frac{G}{2} \sum_{i=1}^{N} \sum_{j=1\\i \ne j}^{N} \frac{m_{i} m_{j}}{r_{ij}} $$

where the sum runs over the $N$ bodies, avoiding the self-interaction case $i=j$ and the factor of \frac{1}{2} is to avoid double counting and the vector $\mathbf{R}$ gives the positions of all bodies - for our example with the Sun, Earth and Mars, $\mathbf{R} = (\mathbf{r}_s, \mathbf{r}_e, \mathbf{r}_m)$.

We will use the convention that vectors denoted by a capital letter (e.g. $\mathbf{R}$) are for the full system, while those denoted by a lower case letter are for an individual body (e.g. $\mathbf{r}_i$). The same convention is applied in code (but without the bold formatting!).

This is an example of a [3-body problem](https://en.wikipedia.org/wiki/Three-body_problem), or more generally, an $N$-body problem.

In [ ]:
D = 2 # dimensionality of space (2D, 3D etc.)
N = 3 # number of bodies - currently Sun, Earth, Mars - we will increase this later

function U(R::AbstractArray)
    r = reshape(R, (N, D)) # convert from 1d vector back to D-dimensional (here 2D)
    U = 0.0
    for i=1:N
        for j=1:N
            (i==j) && continue             # skip self-interation
            r_ij = norm(r[i, :] - r[j, :]) # distance from i to j
            U += -G/2.0 * m[i]*m[j] / r_ij # contribution to potential
        end
    end
    return U
end

### Masses of planetary bodies

For calculating the trajectories of Mars and Earth, we need to know the masses of the Sun, Earth and Mars, as well as their initial positions and velocities. 

The masses of these three bodies are respectively: $1.989 \times 10^{30}$ kg, $5.972 \times 10^{24}$ kg, and $6.39 \times 10^{23}$ kg. Since these numbers are huge, let's scale them by using solar mass units $M_s$, where $M_s = 1.989 \times 10^{30}$ kg. Then, the masses are respectively: $1 M_s$, $0.000003 M_s$, and $0.00000032 M_s$:

In [ ]:
# Masses in solar mass units
m_s = 1.0  
m_e = 0.000003 
m_m = 0.00000032

m = [m_s, m_e, m_m] # pack masses into a 1D array
M = repeat(m, D);   # repeat masses for each dimension

### Distances in astronomical units

Similarily, we scale the distance by using Astronomical Units (A.U.), where 1 A.U. = $1.5 × 10^{11}$ m (which is about the average distance from the Earth to the Sun), and defined the sun to be at coordinates (0,0) in the 2-dimentional plane. The spacecraft is going to be launched on April 1st, 2044, for which the UKSA estimates that the positions of the Sun, Earth and Mars are:

In [ ]:
# Initial position vectors in X,Y coordinates
r_s = [0.0, 0.0]
r_e = [0.1, -1.0] # In astronomical units
r_m = [0.0, -1.524]; # In astronomical units

### Packing and unpacking vectors

To simplify the code for the integration of the equations, it is easier if we pack the positions into an $N\times D$ matrix and then reshape this into a 1D vector of length $ND$.

In [ ]:
@show r0 = [r_s r_e r_m]' # NxD array of positions, NB: transpose

R0 = reshape(r0, N*D) # pack into 1D array
@show R0

# check we can get back to where we started!
@test all(r0 .== reshape(R0, (N, D)))

### Computing forces from the potential

Newton's second law relates the forces felt by each body $i = 1 \ldots N$ to their acceleration:

$$\mathbf{f}_i = m_i \mathbf{a}_i$$

$$-\nabla_i U(\mathbf{R}) = m_i \frac{d \mathbf{v}_i}{dt} = m \frac{d^2\mathbf{r}_i}{dt^2}$$

By taking the derivative of the potential energy with respect to the positions, the force of attraction between the bodies can be calculated via 

$$\mathbf{F} = -\nabla U(\mathbf{R}) = \left(-\frac{\partial U}{\partial x_1}, -\frac{\partial U}{\partial x_2}, \ldots \right) $$. 

Using the `ForwardDiff` package, the potential functions can be defined and their derivatives (forces) calculated without the need of analytic derivation:

In [ ]:
∇U(R) = ForwardDiff.gradient(U, R)

In [ ]:
∇U(R0)

**Lecture Question 2**  Check the automatically computed gradients are correct by comparing with finite-difference gradients computed with a small step size, e.g. $h=10^{-5}$. 

$$\frac{\partial U}{\partial x} \approx \frac{U(x + h) - U(x)}{h}$$

In [ ]:
h = 1e-5
dU = zeros(N*D)
for i=1:N*D
    Rp = copy(R0)
    Rp[i] += h
    dU[i] = (U(Rp) - U(R0)) / h
end

@show maximum(abs, dU .- ∇U(R0));

**Remark.** We have to do this for one variable at a time, which requires $N \times D + 1$ energy evaluations and is too slow to use in real simulations (unlike the automatic differentiation done by `ForwardDiff`). Even better would be to do the differentiation with pen and paper and then code up the derivatives manually.

### Setting the initial velocities

The time units used are years, so the velocity is measured in A.U./year. The initial velocities of the three bodies for the year 2044 are:

In [ ]:
# Initial velocity vectors in X,Y coordinates
v_s = [0.0, 0.0] # In astronomical units per year
v_e = [-6.32, 0.0] # In astronomical units per year
v_m = [-5.05, 0.0] # In astronomical units per year

# pack into NxD array and vector log length N*D
v0 = [v_s v_e v_m]'
V0 = reshape(v0, N*D);

The change in units leads the gravitational constant to change as well. Its new value is:

In [ ]:
G = 37.95  # Gravitational constant

The travel time for the spacecraft to go from Earth to Mars was calculated to be around 1 year. The aliens were expected to arrive in May 2045, so the spacecraft would arrive just in time to defend the human colonies in Mars. 

We will first run our simulation for 2 years to check it's behaviour.

In [ ]:
# Simulation time variables
dt = 0.005
t = 0.0:dt:2
T = length(t) # Total simulation points

## First attempt: Forward Euler integration

If we use the Forward Euler method to integrate the equation of motion we obtain

$$ \mathbf{v}_i(t + \Delta t) = \mathbf{v}_i(t) + \mathbf{a}_i(t) \Delta t $$
$$ \mathbf{r}_i(t + \Delta t) = \mathbf{r}_i(t) + \mathbf{v}_i(t) \Delta t $$

where $\mathbf{r}_i$,  $\mathbf{r}_i$ and $\mathbf{r}_i$ denote the position, velocity and acceleration of body $i$, respectively.  The accelerations are given by $\mathbf{a}_i = -\nabla_i U(\mathbf{R}) / m_i$ from Newton's second law.

We can combine the integration for all the particles using vectors $\mathbf{R}$, $\mathbf{V}$ and $\mathbf{A}$

$$ \mathbf{V}(t + \Delta t) = \mathbf{V}(t) + \mathbf{A}(t) \Delta t $$
$$ \mathbf{R}(t + \Delta t) = \mathbf{R}(t) + \mathbf{V}(t) \Delta t $$


We'll write a function `forward_euler()` to do implement this, and test it on our system. Note how similar the code is to the equations above!

In [ ]:
function forward_euler(∇U, R0, V0, M, dt, T)
    n = length(R0)      # number of degrees of freedom (N x D)
    R = copy(R0)        # current position
    V = copy(V0)        # current velocity
    Rs = zeros(T, n)    # position history (trajectory) 
    Vs = zeros(T, n)    # velocity history

    for i = 1:T
        Rs[i, :] = R    # Store positions
        Vs[i, :] = V    # Store velocities
        
        V_i = copy(V)   # Store current values of velocities
        A = -∇U(R) ./ M # acceleration from Newton's 2nd law (F = ma)
        V += A * dt     # Update velocities
        R += V_i * dt   # Update positions
    end
    return Rs, Vs
end

(R1, V1) = forward_euler(∇U, R0, V0, M, dt, T);

Let's write some code to visualise this trajectory - as usual this ends up quite verbose, but it's not doing anything terribly complicated.

In [ ]:
function plot_trajectory(traj, step=5)
    Rs = reshape(traj, size(traj, 1), N, D)
    
    @gif for n=1:step:size(traj, 1)
               
        # Sun - yellow
        scatter(Rs[1:n, 1, 1], 
                Rs[1:n, 1, 2], 
                color=:yellow, markersize=:30,
                size=(600, 600), xlim=(-2, 2), 
                ylim=(-2, 2), legend=false)
        
        # Earth - blue
        plot!(Rs[1:n, 2, 1], Rs[1:n, 2, 2], color=:blue)
        scatter!([Rs[n, 2, 1]], [Rs[n, 2, 2]], color=:blue, markersize=:20)
        
        # Mars - red
        plot!(Rs[1:n, 3, 1], Rs[1:n, 3, 2], color=:red)
        scatter!([Rs[n, 3, 1]], [Rs[n, 3, 2]], color=:red, markersize=:10)
        
        if size(Rs, 2) > 3
            # for the spaceshop, see later...
            plot!(Rs[1:n, 4, 1], Rs[1:n, 4, 2], color=:green)
            scatter!([Rs[n, 4, 1]], [Rs[n, 4, 2]], color=:green, markersize=:5)
        end
    end
end

We can now plot the trajectories obtained with the Sun in yellow, Earth in blue, Mars in red. 

In [ ]:
plot_trajectory(R1)

Note that there is some drift over time, with the Earth not returning to it's old trajectory in the second year! We either need to reduce the timestep $\Delta t$ or find a more accurate integration scheme.

## Symplectic Euler

The [semi-implicit Euler method](https://en.wikipedia.org/wiki/Semi-implicit_Euler_method) or symplectic Euler method is a very small modification of the Euler method which conserves energy and has much improved performance. All we have to do is update the velocity before using it to update the position - which could come about by a 'bug' in the implementation of the original Euler method!

$$ \mathbf{v}_i(t + \Delta t) = \mathbf{v}_i(t) + \mathbf{a}_i(t) \Delta t $$
$$ \mathbf{r}_i(t + \Delta t) = \mathbf{r}_i(t) + \mathbf{v}_i(t + \Delta t) \Delta t $$


In [ ]:
function symplectic_euler(∇U, R0, V0, M, dt, T)
    n = length(R0)       # number of degrees of freedom (N x D)
    R = copy(R0)         # current position
    V = copy(V0)         # current velocity
    Rs = zeros(T, n)     # position history (trajectory) 
    Vs = zeros(T, n)     # velocity history

    for i = 1:T
        Rs[i, :] = R     # Store positions
        Vs[i, :] = V     # Store velocities
        
        A = -∇U(R) ./ M  # acceleration from Newton's 2nd law (F = ma)
        V += A * dt      # Update velocities
        R += V * dt      # Update positions
    end
    return Rs, Vs
end

(R2, V2) = symplectic_euler(∇U, R0, V0, M, dt, T);

We can reuse our plotting function from the Euler example:

In [ ]:
plot_trajectory(R2)

## Using the `OrdinaryDiffEq` solver package

Instead of integrating the equations of motion ourself, we could use the `OrdinaryDiffEq` package from Lecture 2 to perform a higher-order (more accurate) integration of the equations of motion. We just need to rewrite the second-order equation related positions and acceleration into into a pair of first-order differential equations

$$ \frac{d\mathbf{R}}{dt} = \mathbf{V} $$
$$ \frac{d\mathbf{V}}{dt} = \mathbf{A} = -\frac{\nabla U(\mathbf{R})}{M}$$

In [ ]:
using OrdinaryDiffEq

function f(dx, x, p, t)
    R, V = x[1:N*D], x[N*D+1:end]
    dR = V # derivative of positions are simply the velocities
    dV = -∇U(R) ./ M # derivative of velocities are accelerations    
    dx[1:N*D] = dR
    dx[N*D+1:end] = dV
end

prob = ODEProblem(f, vcat(R0, V0), (0.0, 2.0))
sol = solve(prob, RK4());

To plot the trajectory we first need to convert the ODE solution arrays to the correct shape.

In [ ]:
R_ode = zeros(length(sol), N*D)
for i = 1:length(sol)
    R_ode[i, :] = sol[i][1:N*D]
end
plot_trajectory(R_ode, 1)

Even though the steps are bigger, the use of a higher order integration scheme results in an accurate simulation. We can check that the result at $t=2$ is close to what we got with the symplectic Euler method:

In [ ]:
R_ode[end, :] .- R2[end, :]

If needed, we can interpolate in between the ODE integration points to get a smooth trajectory.

In [ ]:
R_ode_smooth = zeros(T, N*D)
for i = 1:T
    R_ode_smooth[i, :] = sol(t[i])[1:N*D]
end
plot_trajectory(R_ode_smooth, 5)

### Error analysis

If we plot the error in positions as a function of time, assuming the result of the ODE scheme to be exact, we can immediately see the difference between the original and improved Euler  schemes. The original scheme quickly blows up, which the improved scheme oscillates around the correct solution without departing too far from it.

In [ ]:
dR1 = R_ode_smooth .- R1
dR2 = R_ode_smooth .- R2

plot(t, [norm(dR1[i, :]) for i=1:T], label="Foward Euler", 
     xlabel="Time / year", ylabel="Error")
plot!(t, [norm(dR2[i, :]) for i=1:T], label="Symplectic Euler")

We can also look at the conservation of total energy with the two approaches, which shows the same picture.

In [ ]:
U1 = [U(R1[i, :]) for i=1:T] # potential energy
# kinetic energy = Σ 1/2 m v^2
V = reshape(V1, T, N, D)
T1 = [ sum( 1 ./2 * M[j] * norm(V[i, j, :])^2 for j=1:N) for i=1:T]
E1 = U1 + T1 # total energy

U2 = [U(R2[i, :]) for i=1:T] # potential energy
# kinetic energy = Σ 1/2 m v^2
V = reshape(V2, T, N, D)
T2 = [ sum( 1 ./2 * M[j] * norm(V[i, j, :])^2 for j=1:N) for i=1:T]
E2 = U2 + T2 # total energy

plot(t[2:end], E1[2:end], label="Forward Euler", xlabel="Time / year", ylabel="Energy")
plot!(t[2:end], E2[2:end], label="Symplectic Euler")

## Launching a spacecraft from Earth to Mars

Now that the trajectories of Earth and Mars are known, we need to plan the trajectory of the spacecraft. The initial position of the spacecraft is fixed at that of the Earth<sup>*</sup>, but we are free to vary it's initial speed and launch direction.

----

<sup>*</sup> We need to displace the spacecraft slightly away from the Earth, otherwise the distance $r_{ij}$ in the denominator of the equation for potential energy $U$ will be zero, resulting in a diverging energy. The displacement of 1e-3 A.U. is comparable with the radius of the Earth, i.e. as if we launch the rocket from the Earth's surface rather than it's core.

In [ ]:
N = 4 # change number of particles from 3 to 4
D = 2 # still modelling in 2D (x,y) plane

r0 = [r_s r_e r_m]'
r0_spc = [0.1, -1.] .+ 1e-3 # small displacement away from Earth    
r0p = vcat(r0, r0_spc')
R0p = reshape(r0p, N*D)

v0_spc = [6.7, 1.2]
v0 = [v_s v_e v_m]'
v0p = vcat(v0, v0_spc')
V0p = reshape(v0p, N*D)

m_spc = 1e-6

m = [m_s, m_e, m_m, m_spc] 
M = repeat(m, D)

R, V = symplectic_euler(∇U, R0p, V0p, M, dt, T);

In [ ]:
plot_trajectory(R)

Not bad for a first attempt - the spacecraft nearly but not quite meets Mars. We need to fine tune the launch velocity to hit the target. This can be thought of as an optimisation problem -- we could fine-tune the launch velocity by hand, but it would be much quicker to automate it the search.

Let's start by plotting the distance from the spacecraft to Mars as a function of time, and finding the point of closest approach.

In [ ]:
r = reshape(R, T, N, D)
d = [norm(r[i, 3, :] - r[i, 4, :]) for i=1:T] # Mars-spacecraft
d_min, idx_min = findmin(d) # find closest approach of spacecraft to Mars

plot(t, d, label="dist from Mars to spacecraft", xlabel="Time / year", ylabel="Distance / A.U.")
scatter!([t[idx_min]], [d_min], label="", color=:blue)
hline!([0], color=:black, ls=:dash, label="")
vline!([1.0], color=:black, ls=:dash, label="")
ylims!(-0.5, 3)

We want to adjust the launch velocity to move the blue dot to the intersection of the dashed lines, i.e. to make the distance zero after exactly one year. Let's start by wrapping the solver in a function that depends only on the launch speed and angle, with everything else frozen.

In [ ]:
function closest_approach_distance(v, θ)
    r0 = [r_s r_e r_m]'
    r0_spc = [0.1, -1.] .+ 1e-3 # small displacement away from Earth    
    r0p = vcat(r0, r0_spc')
    R0p = reshape(r0p, N*D)
    
    v0_spc = [v*cos(θ), v*sin(θ)]
    v0 = [v_s v_e v_m]'
    v0p = vcat(v0, v0_spc')
    V0p = reshape(v0p, N*D)
    
    m_spc = 1e-6
    m = [m_s, m_e, m_m, m_spc] 
    M = repeat(m, D)
    R3, V3 = symplectic_euler(∇U, R0p, V0p, M, dt, T)
    r3 = reshape(R3, T, N, D)
    d = [norm(r3[i, 3, :] - r3[i, 4, :]) for i=1:T]
    d_min, idx_min = findmin(d)
    t_min = t[idx_min]
    return d_min, t_min, R3
end

We now need to write an *objective function*, i.e. a function that we would like to minimise; in this case it's the distance between the spacecraft and Mars, but we would also like the arrival time to be close to on year from the launch. We can can combine into a single function

$$ f(v, \theta) = w_1 d_\mathrm{min}^2 + w_2 (t_\mathrm{min} - 1)^2 $$

where $w_1$ and $w_2$ are weights that give the relative importance of the two objectives - for now we will set them both to one, but feel free to experiment with this. To fit in with Julia's optimisation routines, we will write this as a function of a single vector quantity $\mathbf{x} = (v, \theta)$.

In [ ]:
# convert from (vx, vy) components to speed and launch angle
vx, vy = 6.7, 1.2
v0 = norm([vx, vy])
θ0 = atan(vy, vx)
x0 = [v0, θ0]

function f(x)
    v, θ = x
    d_min, t_min, R = closest_approach_distance(v, θ)

    w1 = 1.0
    w2 = 0.0
    return w1*d_min^2 + w2*(t_min - 1.0)^2
end

First let's try varying each of these parameters one at a time.

In [ ]:
v = range(6., 8., length=20) # vary launch speed between 6 and 8 A.U/year
θ = range(-π/4, π/4, length=20) # vary launch angle between -45 and 45 degrees

p1 = plot(v, v -> f([v, θ0]), label="Speed v")
p2 = plot(θ, θ -> f([v0, θ]), label="Angle theta / rad")

plot(p1, p2)

It looks like we can do pretty well with just adjusting either the speed or the angle, but we can't quite get $f(v, \theta)$ to zero. Let's look at a two-dimensonal contour plot - we specify the color map to use with the `c` argument, and since we're interested in low values we use `:plasma_r` rather than `plasma` to reverse it.

In [ ]:
contourf(v, θ, (x,y) -> f([x,y]), xlabel="Speed v", ylabel="Angle theta", c=:plasma_r)

## Multi-variate optimisation with the `Optim.jl` package

There looks to be a minimum in the vicinity of $v=7$, $\theta=0.0$. Let's try to find it more precisely using a multi-variable optimisation tool from Julia's `Optim` package, using the [Nelder and Mead simplex method](https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method), which is nice as it doesn't need gradients of our cost function with respect to changes in the parameters.

In [ ]:
using Optim
x0 = [7.0, 0.0]
result = optimize(f, x0, NelderMead())

We can plot where the result lies on our contour map

In [ ]:
v_min, θ_min = result.minimizer

contourf(v, θ, (x,y) -> f([x,y]), xlabel="Speed v", ylabel="Angle theta", c=:plasma_r)
scatter!([v_min], [θ_min], c=:red, label="Optimal solution")

## Optimal trajectory

Finally, let's look at the simulation with these launch parameters to check the rocket really reaches Mars!

In [ ]:
d_min, t_min, R = closest_approach_distance(v_min, θ_min)
plot_trajectory(R)

Oops! What has gone wrong with our assumptions? The challenge continues in **Assignment 4**!